In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
# Standard python import
import sys    # exit
import time   # time accounting
import getopt # command line parser

# --------------------------------------------

# Default settings for command line arguments
DEFAULT_OUTFNAME = "TMVA.root"
DEFAULT_INFNAME  = "tmva_class_example.root"
DEFAULT_TREESIG  = "TreeS"
DEFAULT_TREEBKG  = "TreeB"
DEFAULT_METHODS  = "Cuts,CutsD,CutsPCA,CutsGA,CutsSA,Likelihood,LikelihoodD,LikelihoodPCA,LikelihoodKDE,LikelihoodMIX,PDERS,PDERSD,PDERSPCA,PDEFoam,PDEFoamBoost,KNN,LD,Fisher,FisherG,BoostedFisher,HMatrix,FDA_GA,FDA_SA,FDA_MC,FDA_MT,FDA_GAMT,FDA_MCMT,MLP,MLPBFGS,MLPBNN,CFMlpANN,TMlpANN,SVM,BDT,BDTD,BDTG,BDTB,RuleFit"

# Print usage help
def usage():
    print (" ")
    print ("Usage: python %s [options]") % sys.argv[0]
    print ("  -m | --methods    : gives methods to be run (default: all methods)")
    print ("  -i | --inputfile  : name of input ROOT file (default: '%s')") % DEFAULT_INFNAME
    print ("  -o | --outputfile : name of output ROOT file containing results (default: '%s')") % DEFAULT_OUTFNAME
    print ("  -t | --inputtrees : input ROOT Trees for signal and background (default: '%s %s')") \
          % (DEFAULT_TREESIG, DEFAULT_TREEBKG)
    print ("  -v | --verbose")
    print ("  -? | --usage      : print this help message")
    print ("  -h | --help       : print this help message")
    print (" ")

# Main routine
def main():

    try:
        # retrive command line options
        shortopts  = "m:i:t:o:vh?"
        longopts   = ["methods=", "inputfile=", "inputtrees=", "outputfile=", "verbose", "help", "usage"]
        opts, args = getopt.getopt( sys.argv[1:], shortopts, longopts )

    except getopt.GetoptError:
        # print help information and exit:
        print ("ERROR: unknown options in argument %s") % sys.argv[1:]
        usage()
        sys.exit(1)

    infname     = DEFAULT_INFNAME
    treeNameSig = DEFAULT_TREESIG
    treeNameBkg = DEFAULT_TREEBKG
    outfname    = DEFAULT_OUTFNAME
    methods     = DEFAULT_METHODS
    verbose     = False
    for o, a in opts:
        if o in ("-?", "-h", "--help", "--usage"):
            usage()
            sys.exit(0)
        elif o in ("-m", "--methods"):
            methods = a
        elif o in ("-i", "--inputfile"):
            infname = a
        elif o in ("-o", "--outputfile"):
            outfname = a
        elif o in ("-t", "--inputtrees"):
            a.strip()
            trees = a.rsplit( ' ' )
            trees.sort()
            trees.reverse()
            if len(trees)-trees.count('') != 2:
                print ("ERROR: need to give two trees (each one for signal and background)")
                print (trees)
                sys.exit(1)
            treeNameSig = trees[0]
            treeNameBkg = trees[1]
        elif o in ("-v", "--verbose"):
            verbose = True

    # Print methods
    mlist = methods.replace(' ',',').split(',')
    print ("=== TMVAClassification: use method(s)...")
    for m in mlist:
        if m.strip() != '':
            print ("=== - <%s>") % m.strip()

    # Import ROOT classes
    from ROOT import gSystem, gROOT, gApplication, TFile, TTree, TCut
    
    # check ROOT version, give alarm if 5.18 
    if gROOT.GetVersionCode() >= 332288 and gROOT.GetVersionCode() < 332544:
        print ("*** You are running ROOT version 5.18, which has problems in PyROOT such that TMVA")
        print ("*** does not run properly (function calls with enums in the argument are ignored).")
        print ("*** Solution: either use CINT or a C++ compiled version (see TMVA/macros or TMVA/examples),")
        print ("*** or use another ROOT version (e.g., ROOT 5.19).")
        sys.exit(1)
    
    # Logon not automatically loaded through PyROOT (logon loads TMVA library) load also GUI
    gROOT.SetMacroPath( "./" )
    gROOT.Macro       ( "./TMVAlogon.C" )    
    gROOT.LoadMacro   ( "./TMVAGui.C" )

In [3]:
import ROOT
from ROOT import TMVA, TCut, TFile
from math import floor

Welcome to JupyROOT 6.22/03


In [4]:
outputFile = TFile( "TMVA.root", 'RECREATE' )
TMVA.Tools.Instance()

factory = TMVA.Factory( "TMVAClassification", outputFile,
                       "!V:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification" )

In [5]:
factory.SetVerbose()

Ora dichiariamo un DataLoader e aggiungiamo le variabili usando la AddVariable Function

In [6]:
dataloader = TMVA.DataLoader("dataset")

In [7]:
dataloader.AddVariable( "massK0S", 'F' )
dataloader.AddVariable( "tImpParBach", 'F' )
dataloader.AddVariable( "tImpParV0", 'F' )
dataloader.AddVariable( "CtK0S := DecayLengthK0S*0.497/v0P", 'F')
dataloader.AddVariable( "cosPAK0S", 'F' )
dataloader.AddVariable( "CosThetaStar", 'F' )
dataloader.AddVariable( "nSigmaTOFpr", 'F' )
dataloader.AddVariable( "nSigmaTOFpi", 'F' )
dataloader.AddVariable( "nSigmaTOFka", 'F' )
dataloader.AddVariable( "nSigmaTPCpr", 'F' )
dataloader.AddVariable( "nSigmaTPCpi", 'F' )
dataloader.AddVariable( "nSigmaTPCka", 'F' )

#spectator variables

dataloader.AddSpectator( "massLc2K0Sp", "mass Lc-->K0S+p", 'F' )
dataloader.AddSpectator( "LcPt",  "Lc Pt", 'F' )
dataloader.AddSpectator( "massLc2Lambdapi",  "mass Lc -->L(1520)+pi", 'F' )
dataloader.AddSpectator( "massLambda", "mass V0 = Lambda", 'F' )
dataloader.AddSpectator( "massLambdaBar", "mass V0 = LambdaBar", 'F' )
dataloader.AddSpectator( "V0positivePt", "V0 positive Pt", 'F' )
dataloader.AddSpectator( "V0negativePt", "V0 negative Pt", 'F' )
dataloader.AddSpectator( "dcaV0pos", "dca V0 positive", 'F' )
dataloader.AddSpectator( "dcaV0neg", "dca V0 negative", 'F' )
dataloader.AddSpectator( "v0Pt", "K0S Pt", 'F' )
dataloader.AddSpectator( "dcaV0", "dca V0", 'F' )
dataloader.AddSpectator( "V0positiveEta", "V0pos Eta", 'F' )
dataloader.AddSpectator( "bachelorEta", "eta bachelor", 'F' )

apriamo il dataset contenente signal and background trees

In [8]:
input= TFile.Open( "AnalysisResults.root")

Warning in <TClass::Init>: no dictionary for class AliRDHFCutsLctoV0 is available
Warning in <TClass::Init>: no dictionary for class AliRDHFCuts is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisCuts is available
Warning in <TClass::Init>: no dictionary for class AliVCuts is available
Warning in <TClass::Init>: no dictionary for class AliESDtrackCuts is available
Warning in <TClass::Init>: no dictionary for class AliAODPidHF is available
Warning in <TClass::Init>: no dictionary for class AliAnalysisUtils is available
Warning in <TClass::Init>: no dictionary for class AliEventCuts is available
Warning in <TClass::Init>: no dictionary for class AliNormalizationCounter is available
Warning in <TClass::Init>: no dictionary for class AliCounterCollection is available


In [9]:
# signal and backg trees for training
signal      = input.Get( "treeList_2_25_2_25_Sgn")
background  = input.Get( "treeList_2_25_2_25_Bkg")

global event weights per tree (see below for setting event-wise weights) Diamo al Training e al Test lo stesso tree, questo verrà quindi diviso in due sottounità che serviranno separatamente per il training e il test.

In [10]:
signalWeight     = 1.0
backgroundWeight = 1.0

dataloader.AddSignalTree(signal, signalWeight)
dataloader.AddBackgroundTree(background, backgroundWeight)

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree treeList_2_25_2_25_Sgn of type Signal with 1762028 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree treeList_2_25_2_25_Bkg of type Background with 1473789 events


aggiungo dei Tagli sull'impulso trasverso per selezionare solo una parte degli eventi presenti nel tree

In [11]:
mycuts = TCut("LcPt<12 && LcPt>8")
mycutb = TCut("LcPt<12 && LcPt>8")

Applicato il taglio sul momento trasverso il numero di eventi da utilizzare nel training e nel test viene deciso come segue:

1) il training e il testing del segnale viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 100000

2) il training e il testing del Bakground viene fatto con il valore minimo tra la metà degli eventi rimasti dopo il taglio e 200000

Training e testing vengono ovviamente fatti con eventi diversi selezionati in maniera randomica all'interno del tree


In [12]:
nTrainingEventsSgn = floor(min(signal.GetEntries("LcPt<12 && LcPt>8")*0.5,100000))
nTrainingEventsBkg = floor(min(background.GetEntries("LcPt<12 && LcPt>8")*0.5,200000))
nTestingEventsSgn = floor(min(signal.GetEntries("LcPt<12 && LcPt>8")*0.5,100000))
nTestingEventsBkg = floor(min(background.GetEntries("LcPt<12 && LcPt>8")*0.5,200000))

71245


Usiamo la funzione DataLoader.PrepareTrainingAndTestTree per applicare i cut agli eventi di input

In [13]:
dataloader.PrepareTrainingAndTestTree( mycuts, mycutb,
"nTrain_Signal=nTrainingEventsSgn:nTest_Signal=nTestingEventsSgn:nTrain_Background=nTrainingEventsBkg:nTest_Background=nTestingEventsBkg:SplitMode=Random:NormMode=NumEvents:!V")


In [14]:
factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDTG","!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2" );

Factory                  : Booking method: BDTG
                         : 
                         : the option NegWeightTreatment=InverseBoostNegWeights does not exist for BoostType=Grad
                         : --> change to new default NegWeightTreatment=Pray
                         : Building event vectors for type 2 Signal
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Sgn
                         : Building event vectors for type 2 Background
                         : Dataset[dataset] :  create input formulas for tree treeList_2_25_2_25_Bkg
DataSetFactory           : [dataset] : Number of events in input trees
                         : Dataset[dataset] :     Signal     requirement: "LcPt<12 && LcPt>8"
                         : Dataset[dataset] :     Signal          -- number of events passed: 142491  / sum of weights: 142491
                         : Dataset[dataset] :     Signal          -- efficiency             : 0.080

In [16]:
factory.BookMethod(dataloader, TMVA.Types.kMLP, "MLP", "H:!V:NeuronType=tanh:VarTransform=N:NCycles=600:HiddenLayers=N+5:TestRate=5:!UseRegulator" )

Factory                  : Booking method: MLP
                         : 
MLP                      : [dataset] : Create Transformation "N" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'massK0S' <---> Output : variable 'massK0S'
                         : Input : variable 'tImpParBach' <---> Output : variable 'tImpParBach'
                         : Input : variable 'tImpParV0' <---> Output : variable 'tImpParV0'
                         : Input : variable 'CtK0S' <---> Output : variable 'CtK0S'
                         : Input : variable 'cosPAK0S' <---> Output : variable 'cosPAK0S'
                         : Input : variable 'CosThetaStar' <---> Output : variable 'CosThetaStar'
                         : Input : variable 'nSigmaTOFpr' <---> Output : variable 'nSigmaTOFpr'
                         : Input : variable 'nSigmaTOFpi' <---> Output : variable 'nSigmaTOFp

In [17]:
# Train MVAs
factory.TrainAllMethods()

Factory                  : Train all methods
Factory                  : [dataset] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'massK0S' <---> Output : variable 'massK0S'
                         : Input : variable 'tImpParBach' <---> Output : variable 'tImpParBach'
                         : Input : variable 'tImpParV0' <---> Output : variable 'tImpParV0'
                         : Input : variable 'CtK0S' <---> Output : variable 'CtK0S'
                         : Input : variable 'cosPAK0S' <---> Output : variable 'cosPAK0S'
                         : Input : variable 'CosThetaStar' <---> Output : variable 'CosThetaStar'
                         : Input : variable 'nSigmaTOFpr' <---> Output : variable 'nSigmaTOFpr'
                         : Input : variable 'nSigmaTOFpi' <---> Output : variable 'nSigmaTOFpi'
                         : 

0%, time left: unknown
6%, time left: 70 sec
12%, time left: 60 sec
18%, time left: 53 sec
25%, time left: 48 sec
31%, time left: 44 sec
37%, time left: 41 sec
43%, time left: 37 sec
50%, time left: 33 sec
56%, time left: 29 sec
62%, time left: 25 sec
68%, time left: 20 sec
75%, time left: 16 sec
81%, time left: 12 sec
87%, time left: 8 sec
93%, time left: 4 sec
0%, time left: unknown
6%, time left: 6 sec
12%, time left: 5 sec
18%, time left: 5 sec
25%, time left: 4 sec
31%, time left: 4 sec
37%, time left: 3 sec
43%, time left: 3 sec
50%, time left: 3 sec
56%, time left: 2 sec
62%, time left: 2 sec
68%, time left: 2 sec
75%, time left: 1 sec
81%, time left: 1 sec
87%, time left: 0 sec
93%, time left: 0 sec
0%, time left: unknown
6%, time left: 9 mins
12%, time left: 8 mins
19%, time left: 8 mins
25%, time left: 7 mins
31%, time left: 7 mins
37%, time left: 6 mins
44%, time left: 5 mins
50%, time left: 5 mins
56%, time left: 274 sec
62%, time left: 237 sec
69%, time left: 198 sec
75%, 

In [18]:
# Test MVAs
factory.TestAllMethods()

Factory                  : Test all methods
Factory                  : Test method: BDTG for Classification performance
                         : 
BDTG                     : [dataset] : Evaluation of BDTG on testing sample (86244 events)
                         : Elapsed time for evaluation of 86244 events: 4.52 sec       
Factory                  : Test method: MLP for Classification performance
                         : 
MLP                      : [dataset] : Evaluation of MLP on testing sample (86244 events)
                         : Elapsed time for evaluation of 86244 events: 0.366 sec       


0%, time left: unknown
6%, time left: 4 sec
12%, time left: 4 sec
18%, time left: 3 sec
25%, time left: 3 sec
31%, time left: 2 sec
37%, time left: 2 sec
43%, time left: 2 sec
50%, time left: 2 sec
56%, time left: 2 sec
62%, time left: 1 sec
68%, time left: 1 sec
75%, time left: 1 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec
0%, time left: unknown
6%, time left: 0 sec
12%, time left: 0 sec
18%, time left: 0 sec
25%, time left: 0 sec
31%, time left: 0 sec
37%, time left: 0 sec
43%, time left: 0 sec
50%, time left: 0 sec
56%, time left: 0 sec
62%, time left: 0 sec
68%, time left: 0 sec
75%, time left: 0 sec
81%, time left: 0 sec
87%, time left: 0 sec
93%, time left: 0 sec


In [19]:
# Evaluate MVAs
factory.EvaluateAllMethods() 

Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: BDTG
                         : 
BDTG                     : [dataset] : Loop over test events and fill histograms with classifier response...
                         : 
TFHandler_BDTG           :     Variable            Mean            RMS    [        Min            Max ]
                         : -------------------------------------------------------------------------------
                         :      massK0S:       0.49807     0.0039421   [       0.48761       0.50761 ]
                         :  tImpParBach:   -0.00031003      0.028152   [      -0.49872       0.49947 ]
                         :    tImpParV0:     0.0010052      0.091539   [       -1.4706        1.4860 ]
                         :        CtK0S:        2.6764        2.4028   [      0.042344        118.06 ]
                         :     cosPAK0S:       0.99991    0.00052306   [       0.99000        1.0000 ]
        

In [20]:
# Save the output.
outputFile.Close()